In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
from numpy.typing import ArrayLike

In [2]:
from rl_projects.envs.cartpole import get_default_env, random_policy, render_episode

env = get_default_env()

In [3]:
from typing import Callable

import numpy as np
import torch
import torch.nn as nn

from rl_projects.drl.utils import MLP, EpsilonGreedyChoice
from rl_projects.drl.nfq import NFQAgent
from tqdm.notebook import tqdm

agent = NFQAgent(
    env,
    MLP(input_dim=env.observation_space.shape[0], output_dim=env.action_space.n, hidden_dims=[512, 128]),
    action_choice=EpsilonGreedyChoice(0.5)
)

agent.train(env, n_episodes=1000, n_epochs=40)

Episode:   0%|          | 0/1000 [00:00<?, ?it/s]